In [ ]:
# =============================================
# Fair Exchange Value Predictor (Tea/Coffee Credits)
# =============================================

# 1️⃣ Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings("ignore")

# =============================================
# 2️⃣ Simulate Example Dataset
# =============================================

np.random.seed(42)
num_samples = 500

# Features
data = {
    'skill_rarity': np.random.uniform(0.1, 1.0, num_samples),        # 0 (common) to 1 (rare)
    'demand_score': np.random.uniform(0.1, 1.0, num_samples),        # 0 (low demand) to 1 (high)
    'mentor_experience': np.random.uniform(0, 10, num_samples),      # years of experience
    'feedback_score': np.random.uniform(1, 5, num_samples),          # average feedback (1–5)
    'duration_hours': np.random.uniform(0.5, 3.0, num_samples),      # session duration in hours
}

df = pd.DataFrame(data)

# Target: Fair Credit Value (nonlinear relation)
df['chai_credit_value'] = (
    1.5 * df['skill_rarity'] +
    1.2 * df['demand_score'] +
    0.3 * df['mentor_experience'] / 10 +
    0.5 * df['feedback_score'] / 5 +
    0.7 * df['duration_hours'] +
    np.random.normal(0, 0.1, num_samples)
) * 2.5  # scaled credit system

print("Sample Data:\n", df.head())

# =============================================
# 3️⃣ Split Data
# =============================================
X = df.drop(columns=['chai_credit_value'])
y = df['chai_credit_value']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =============================================
# 4️⃣ Feature Scaling
# =============================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# =============================================
# 5️⃣ Random Forest Model
# =============================================
rf_model = RandomForestRegressor(n_estimators=150, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)

# =============================================
# 6️⃣ XGBoost Model
# =============================================
xgb_model = XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    random_state=42
)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)

# =============================================
# 7️⃣ Neural Network Model
# =============================================
nn_model = Sequential([
    Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

nn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')
nn_model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, verbose=0)
nn_preds = nn_model.predict(X_test_scaled).flatten()

# =============================================
# 8️⃣ Evaluation
# =============================================
def evaluate_model(name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"\n{name} Model Performance:")
    print(f"  Mean Absolute Error: {mae:.3f}")
    print(f"  R² Score: {r2:.3f}")

evaluate_model("Random Forest", y_test, rf_preds)
evaluate_model("XGBoost", y_test, xgb_preds)
evaluate_model("Neural Network", y_test, nn_preds)

# =============================================
# 9️⃣ Sample Prediction
# =============================================
sample = pd.DataFrame({
    'skill_rarity': [0.8],
    'demand_score': [0.9],
    'mentor_experience': [5],
    'feedback_score': [4.5],
    'duration_hours': [1.5]
})

# Predict fair chai credits
sample_scaled = scaler.transform(sample)
print("\nSample Input:\n", sample)
print("\nPredicted Fair Credit Values:")
print("  Random Forest:", rf_model.predict(sample)[0])
print("  XGBoost:", xgb_model.predict(sample)[0])
print("  Neural Network:", nn_model.predict(sample_scaled)[0])
